# Benchmarking

In [1]:
import sys
import os
import time
sys.path.append('/Users/calum/Developer/pyzx-heuristics-master')
import pyzx as zx_heuristics
sys.path.append('..')
# import pyzx as zx
from benchmarking import benchmark

In [2]:
%%script false --no-raise-error
b = benchmark(dirpath='benchmark')
b.show_attributes()

In [3]:
# %%script false --no-raise-error
b = benchmark()
b.load_circuits(os.path.join('..', 'circuits', 'benchmarking', 'Fast', 'before'), group_name='fast')
b.load_circuits(os.path.join('..', 'circuits', 'benchmarking', 'Fast', 'nrscm'), group_name='fast', simp_strategy='NRSCM')

In [4]:
base_params = [1,1,1,0,0,0,0,0,0]
cFlow_params = [0.37,0.4,0.23,-0.94,-0.71,0.5,-0.45,0.47,0.59]

def cFlow_reduce(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce(g,x=base_params, quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_simple(g2, up_to_perm=True).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def gFlow_reduce(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce(g,x=base_params, flow = 'g', quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_circuit(g2, up_to_perm=True).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def zx_heur(c):
    t0 = time.time()
    g = c.to_graph()
    g = zx_heuristics.simplify.teleport_reduce(g)
    g.track_phases = False
    zx_heuristics.simplify.greedy_simp_neighbors(g)
    t1 = time.time()
    c2 = zx_heuristics.extract_circuit(g, up_to_perm=True).to_basic_gates()
    c3 = zx_heuristics.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx_heuristics.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t1-t0, t2-t0
    return c3, t1-t0, t2-t0

def basic(c):
    c1 = zx.optimize.basic_optimization(c.copy(), do_swaps=False).to_basic_gates()
    c2 = zx.optimize.basic_optimization(c.copy(), do_swaps=True).to_basic_gates()
    if c2.twoqubitcount() < c1.twoqubitcount(): return c2
    return c1

In [5]:
b.add_simplification_func(zx_heur, 'Heur', groups_to_run=['fast'],verify=False)
# b.add_simplification_func(cFlow_reduce, 'cFlow', groups_to_run=['fast'],verify=False)
# b.add_simplification_func(gFlow_reduce, 'gFlow', groups_to_run=['fast'],verify=False)
# b.add_simplification_func(basic, 'Basic', groups_to_run=['fast'],verify=False)

Processing Heur on qft_4                                              : 100%|██████████| 32/32 [08:48<00:00, 16.52s/it]


In [6]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['Heur']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,Heur,NRSCM
fast,Y,Y,Y


In [7]:
df = b.df(groups=['fast'],
     routines=['NRSCM'],
     funcs='all',
     atts=['Qubits', '2Q Count', 'T Count', 't_simp', 't_opt'])

In [18]:
# funcs = ['full_reduce_flow_b', 'full_reduce_min_flow_b', 'basic_optimize', 'clifford_simp_b', 'clifford_simp_flow_b', 'clifford_simp_min_flow_b', 'full_reduce_b']
# t_probs = [0.015*i for i in range(11)]
# fig = b.Pt_graphs(funcs=funcs, qubits=8, depth=800, cnot_prob=0.3, t_probs=t_probs, ys=['Gates','2Q Count','T Count'], reps=20, overwrite=False, random_seed=42)

In [13]:
b.save(dirpath='benchmark')